(ch_if_then_else)=
# 条件表达式：`if-then-else`

通过 `te.if_then_else` 支持 `if-then-else` 语句。在本节中，将以计算矩阵的下三角形为例介绍这个表达式。

In [1]:
# from tvm_book.contrib import d2ltvm
import numpy as np
import tvm
from tvm import te

在 NumPy 中，可以很容易地使用 `np.tril` 得到下三角形。

In [2]:
a = np.arange(12, dtype='float32').reshape((3, 4))
np.tril(a)

array([[ 0.,  0.,  0.,  0.],
       [ 4.,  5.,  0.,  0.],
       [ 8.,  9., 10.,  0.]], dtype=float32)

在 TVM 中使用 `if_then_else` 实现它。它接受三个参数，第一个是条件，如果为真返回第二个参数，否则返回第三个参数。

In [3]:
n, m = te.var('n'), te.var('m')
A = te.placeholder((n, m))
B = te.compute(A.shape,
               lambda i, j: te.if_then_else(i >= j, A[i, j], 0.0))


验证结果。

In [4]:
b = tvm.nd.array(np.empty_like(a))
s = te.create_schedule(B.op)
ir_mod = tvm.lower(s, [A, B], simple_mode=True)
mod = tvm.build(ir_mod)
mod(tvm.nd.array(a), b)
b

<tvm.nd.NDArray shape=(3, 4), cpu(0)>
array([[ 0.,  0.,  0.,  0.],
       [ 4.,  5.,  0.,  0.],
       [ 8.,  9., 10.,  0.]], dtype=float32)

In [5]:
ir_mod["main"]

PrimFunc([placeholder, compute]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (i, 0, n) {
    for (j, 0, m) {
      compute[((i*stride) + (j*stride))] = tir.if_then_else((j <= i), placeholder[((i*stride) + (j*stride))], 0f)
    }
  }
}

## 小结

- 可以用 `tvm.if_then_else` 用于 if-then-else 语句。